In [ ]:
!pip install -q  langchain tqdm

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.docstore.document import Document as LangchainDocument
from tqdm.auto import tqdm
import random
import pandas as pd
import numpy as np
pd.set_option("display.max_colwidth", None)

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:

import torch
from torch.nn import functional as F
import numpy as np
from abc import ABC, abstractmethod
from typing import List, Tuple, Dict
from transformers import AutoModel, AutoTokenizer
from embedding_models import EmbeddingModelMiniLML6


IR evaluation (EmbeddingModelMiniLML6)

In [ ]:
from tqdm import tqdm
import numpy as np
import json
import torch


model = EmbeddingModelMiniLML6()

with open(f"data_raw.json", 'r') as f:
    raw_data = json.load(f)
    metadata_list = []
    for i in range(len(raw_data['abstracts'])):
        metadata_list.append({
            "abstract": raw_data['abstracts'][i],
            "authors": raw_data['authior'][i],
            "title": raw_data['title'][i],
            "article_id": raw_data['article_id'][i]
        })

print("Found", len(metadata_list), "samples in 'data_raw.json'")
print("Embedding chunks..")

data = []
chunks_list=[]
for sample in tqdm(metadata_list):
    embedded_chunks = model.embed_passage(sample['abstract'])

    for chunk in embedded_chunks:
        data.append({
            "embedding": chunk,
            "metadata": sample
        })
        chunks_list.append(sample['abstract'])
print("Produced", len(data), "embedded chunks together with their metadata")
print("Saving to data.json..")

with open("database_numpy.json", "w+") as f:
    f.write(json.dumps(data, indent=2))

print("Finished!")

Found 874 samples in 'data_raw.json'
Embedding chunks..


100%|██████████| 874/874 [02:48<00:00,  5.19it/s]


Produced 903 embedded chunks together with their metadata
Saving to data.json..
Finished!


In [ ]:
from huggingface_hub import InferenceClient


repo_id = "mistralai/Mixtral-8x7B-Instruct-v0.1"

llm_client = InferenceClient(
    model=repo_id,
    timeout=120,
)


def call_llm(inference_client: InferenceClient, prompt: str):
    response = inference_client.post(
        json={
            "inputs": prompt,
            "parameters": {"max_new_tokens": 1000},
            "task": "text-generation",
        },
    )
    return json.loads(response.decode())[0]["generated_text"]


call_llm(llm_client, "This is a test context")

'This is a test context for the `@mui/material` library.\n\n## Installation\n\n```sh\nnpm install @mui/material\n```\n\n## Usage\n\n```jsx\nimport React from \'react\';\nimport { Button } from \'@mui/material\';\n\nfunction App() {\n  return (\n    <div className="App">\n      <Button variant="contained" color="primary">\n        Hello World\n      </Button>\n    </div>\n  );\n}\n\nexport default App;\n```\n\n## Documentation\n\n- [Material-UI](https://material-ui.com/)\n- [Material Design](https://material.io/)'

In [ ]:
QA_generation_prompt = """
Your task is to write a factoid question and an answer given a context.
Your factoid question should be answerable with a specific, concise piece of factual information from the context.
Your factoid question must be highly relevant to the given context, so that answer can be obtained from the context,  and must be a medical question.

Your factoid question should ideally include proper nouns from the given context.
Your factoid question should be formulated in the same style as questions users could ask in a search engine.
This means that your factoid question MUST NOT mention something like "according to the passage" or "context".

Provide your answer as follows:

Output:::
Factoid question: (your factoid question)
Answer: (your answer to the factoid question)

Now here is the context.

Context: {context}\n
Output:::"""

In [ ]:
import random
import json


N_GENERATIONS = 250
print(f"Generating {N_GENERATIONS} QA couples...")

outputs = {"context":[],"question":[],"answer":[]}
for sampled_context in tqdm(random.sample(chunks_list, N_GENERATIONS)):
    # Generate QA couple
    output_QA_couple = call_llm(
        llm_client, QA_generation_prompt.format(context=sampled_context)
    )
    try:
        question = output_QA_couple.split("Factoid question: ")[-1].split("Answer: ")[0]
        answer = output_QA_couple.split("Answer: ")[-1]
        assert len(answer) < 300, "Answer is too long"
        outputs["context"].append(sampled_context)
        outputs["question"].append(question)
        outputs["answer"].append(answer)
        with open("QA_Generated_EmbeddingModelMiniLML6.json", "w") as f:
                    json.dump(outputs, f)

    except:
        continue


Generating 250 QA couples...


100%|██████████| 250/250 [12:46<00:00,  3.07s/it]


In [ ]:
display(pd.DataFrame(outputs).head(5))

,context,question,answer
0,"Dopamine (DA) is considered to be an important neurotransmitter in the control of impulsive behavior, however, its underlying mechanisms have not been fully elucidated. Catechol-O-methyltransferase (COMT) is a key enzyme in the catabolism of DA within the prefrontal cortex (PFC) and has been suggested to play a role in the mediation of impulsive behavior. The COMT single nucleotide polymorphism (SNP) rs4680 (Val158Met) Met allele has been shown to decrease COMT enzyme activity and is associated with improved PFC cognitive function (intelligence and executive functions). Studies have associated the rs4680 genotype with impulsivity as a symptom in attention deficit hyperactivity disorder and substance abuse. However, only a few studies have assessed the effects of rs4680 on impulsiveness in healthy subjects, the results of which remain controversial. The Barratt Impulsiveness Scale (BIS-11) was applied to 82 healthy volunteers (including 42 females) who were genotyped for COMT rs4680. Subjects carrying the Met/Met genotype scored higher for the BIS-11 second-order factor Non-planning than carriers of the Val/Val genotype. No interaction between gender genotype was detected. Age, gender and education had no effect on the results. The COMT rs4680 Met/Met genotype was associated with higher impulsivity on the BIS-11 second-order factor Non-planning. These results suggest that COMT enzyme activity may be important in the regulation of impulsiveness among young adults. Further studies involving larger samples should be conducted to confirm the results of the present study.",Is there a relationship between the COMT rs4680 genotype and impulsiveness in young adults?\n,"Yes, young adults carrying the COMT rs4680 Met/Met genotype scored higher for the BIS-11 second-order factor Non-planning, indicating a higher level of impulsiveness."
1,"An inverse relationship between blood pressure (BP) and cognitive function has been found in adults, but limited data are available in adolescents and young adults. We examined the prospective relation between BP and cognitive function in adolescence.\nWe examined the association between BP measured at the ages of 12-15 years in school surveys and cognitive endpoints measured in the Seychelles Child Development Study at ages 17 (n = 407) and 19 (n = 429) years, respectively. We evaluated multiple domains of cognition based on subtests of the Cambridge Neurological Test Automated Battery (CANTAB), the Woodcock Johnson Test of Scholastic Achievement (WJTA), the Finger Tapping test (FT) and the Kaufman Brief Intelligence Test (K-BIT). We used age, sex and height-specific z-scores of SBP, DBP and mean arterial pressure (MAP).\nSix out of the 21 cognitive endpoints tested were associated with BP. However, none of these associations were found to hold for both males and females or for different subtests within the same neurodevelopmental domain or for both SBP and DBP. Most of these associations disappeared when analyses were adjusted for selected potential confounding factors such as socio-economic status, birth weight, gestational age, BMI, alcohol consumption, blood glucose, and total n-3 and n-6 polyunsaturated fats.\nOur findings do not support a consistent association between BP and subsequent performance on tests assessing various cognitive domains in adolescents.",Is there an association between blood pressure and cognitive function in adolescents?\n,"No, the study found no consistent association between blood pressure and subsequent performance on tests assessing various cognitive domains in adolescents."
2,"Genetic studies of single gene variants have been criticized as providing a simplistic characterization of the genetic basis of illness risk that ignores the effects of other variants within the same biological pathways. Of candidate biological pathways for schizophrenia (SZ), the cell adhesion molecule (CAM) pathway has repeatedly been linked to both psychosis and neu

In [ ]:

data=[]
from google.colab import drive
drive.mount('/content/drive')

with open(f"/content/drive/MyDrive/valid_abstracts2013.json", 'r') as f:
      data_annual = json.load(f)
      data.extend(data_annual["abstracts"])

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Use our chunks to generate QA couples

In [ ]:
#To generate chunks, need to be deleted when we use our chunks
!pip install sentence-transformers
from langchain.text_splitter import SentenceTransformersTokenTextSplitter
langchain_docs = [
    LangchainDocument(page_content=text)
    for text in tqdm(data)
]

embed_model_id = "sentence-transformers/all-mpnet-base-v2"

text_splitter = SentenceTransformersTokenTextSplitter(
    model_name=embed_model_id,
    chunk_overlap=10,

)


  0%|          | 0/874 [00:00<?, ?it/s]

In [ ]:
#Store chunk in chunks_eval_list
chunks_eval_list=[]
for i in range(len(data)):

    chunks_eval_list += text_splitter.split_text(text=data[i])


In [ ]:


N_GENERATIONS = 250
print(f"Generating {N_GENERATIONS} QA couples...")

outputs1 = {"context":[],"question":[],"answer":[]}
for sampled_context in tqdm(random.sample(chunks_list, N_GENERATIONS)):
    # Generate QA couple
    output_QA_couple = call_llm(
        llm_client, QA_generation_prompt.format(context=sampled_context)
    )
    try:
        question = output_QA_couple.split("Factoid question: ")[-1].split("Answer: ")[0]
        answer = output_QA_couple.split("Answer: ")[-1]
        assert len(answer) < 300, "Answer is too long"
        outputs1["context"].append(sampled_context)
        outputs1["question"].append(question)
        outputs1["answer"].append(answer)
        with open("QA_Generated_Pinecone.json", "w") as f:
                    json.dump(outputs1, f)

    except:
        continue

Generating 250 QA couples...


  0%|          | 0/250 [00:00<?, ?it/s]

In [ ]:
display(pd.DataFrame(outputs).head(5))

,context,question,answer
0,"), especially on the factors : work ( p = 0. 032 ) ; extended family ( p = 0. 005 ). higher novelty seeking ( ns ) scores were significantly correlated with worse on global ( p = 0. 002 ) ; work ( p = 0. 001 ) and leisure ( p = 0. 003 ). there was no correlation between cognitive performance and social adjustment scale ( sas ) factors. higher seizure frequency - - myoclonic ( p = 0. 005 ) and gtc ( p = 0. 035 ) - - were correlated with higher scores on factor work of sas. our findings suggest that patients with jme have worse social adjustment in two relevant aspects of their lives - - work and familiar relationship. in this series of patients with jme, higher seizure frequency and impulsive traits, but not cognitive performance, were correlated with worse social adjustment.",What type of seizure frequency is correlated with higher scores on factor work of the social adjustment scale in patients with JME?\n,Myoclonic and GTC seizure frequency are correlated with higher scores on factor work of the social adjustment scale in patients with JME.
1,of sleep pressure during weekdays. interventions that target the misalignment of endogenous circadian rhythms and imposed rhythms are discussed.,What is sleep pressure discussed in the context?\n,Sleep pressure during weekdays is discussed in the context.
2,"autism spectrum disorders ( asd ) are characterised by abnormalities in social interaction and communication skills, as well as stereotypic behaviours and restricted activities and interests. selective serotonin reuptake inhibitors ( ssris ) are prescribed for the treatment of conditions often comorbid with asd such as depression, anxiety and obsessive - compulsive behaviours. to determine if treatment with an ssri : 1. improves the core features of autism ( social interaction, communication and behavioural problems ) ; 2. improves other non - core aspects of behaviour or function such as self - injurious behaviour ; 3. improves the quality of life of adults or children and their carers ; 4. has short - and long - term effects on outcome ; 5. causes harm. we searched the following databases up until march 2013 : central, ovid medline, embase, cinahl, psycinfo, eric and sociological abstracts. we also searched clinicaltrials. gov and the international clinical trials registry platform ( ictrp ). this was supplemented by searching reference lists and contacting known experts in the field. randomised controlled trials ( rcts ) of any dose of oral ssri compared with placebo, in people with asd. two authors independently selected studies for inclusion, extracted data and appraised each study's risk of bias. nine rcts with a total of 320 participants were included. four ssris were evaluated : fluoxetine ( three studies ), fluvoxamine ( two studies ), fenfluramine ( two studies ) and citalopram ( two studies ). five studies included only children and four studies included only adults. varying inclusion criteria were used with regard to diagnostic criteria and intelligence quotient of participants. eighteen different outcome measures were reported. although more than one study reported data for clinical global impression ( cgi ) and ob",Does fluoxetine improve the core features of autism spectrum disorders?\n,"According to the passage, fluoxetine has been studied in three randomized controlled trials, but the passage does not provide information on whether it improves the core features of autism spectrum disorders."
3,"both impaired insight and cognitive deficits in schizophrenia are core features of this disorder. previous studies have demonstrated the complex relationships between neurocognition and cognitive insight, as well as the contribution of neurocognition in explaining cognitive insight. however, there is lack of research regarding the influences of sex on the relation of neurocognition and cognitive insight. the present study sought to elucidate sex differences in cognitive insight and neurocognition i